In [132]:
#!/usr/bin/env python3
import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# Record the start time
start_time = time.perf_counter()
print("Starting analysis...")
HZ4Lep = uproot.open("hz_hww_4l.root")
HZ4Lep_tree = HZ4Lep["Delphes"]
column_arrays = ["MissingET_size","MissingET.Eta","MissingET.Phi","Jet_size","Electron_size","Electron.Eta","Electron.Phi","Muon_size","Muon.Eta","Muon.Phi"]
HZ4Lep_array = HZ4Lep_tree.arrays(column_arrays, library="np", entry_stop=13)

Starting analysis...


In [134]:

HZ4Lep_array[1]

KeyError: 1

In [130]:
def check_drFromMET(n_array, name):
    # Calculate delta R between MET and each lepton
    fig=plt.figure()
    ax=fig.add_subplot(111)
    ax.set_title(f"Delta R from MET to Leptons - {name}")
    ax.set_xlabel("Delta R")
    ax.set_ylabel("Count")
    deta_r=np.array([])
    for number_event in range(len(n_array)):
        enum=n_array["Electron_size"][number_event]
        for ie in range(enum):
            dphi=n_array["Electron.Phi"][number_event][ie]-n_array["MissingET.Phi"][number_event]
            while (dphi > np.pi):
                dphi -= 2*np.pi
            while (dphi < -np.pi):
                dphi += 2*np.pi
            dr=np.sqrt((n_array["Electron.Eta"][number_event][ie]-n_array["MissingET.Eta"][number_event])**2 + dphi**2)
            deta_r=np.append(deta_r, dr)
        mnum=n_array["Muon_size"][number_event]
        for im in range(mnum):
            dphi=n_array["Muon.Phi"][number_event][im]-n_array["MissingET.Phi"][number_event]
            while (dphi > np.pi):
                dphi -= 2*np.pi
            while (dphi < -np.pi):
                dphi += 2*np.pi
            dr=np.sqrt((n_array["Muon.Eta"][number_event][im]-n_array["MissingET.Eta"][number_event])**2 + dphi**2)
            deta_r=np.append(deta_r, dr) 
            
    ax.hist(deta_r, bins=100, range=(0, 5))

    plt.savefig(f"DeltaR_MET_Leptons_{name}.png")
    plt.close()
    return 1
check_drFromMET(HZ4Lep_array, "HZ4Lep")

1